In [2]:
# ================= Multi-target, multi-model AUC benchmarking (entry-only features)
import os, re, inspect, warnings  # stdlib
import numpy as np, pandas as pd  # core
import matplotlib.pyplot as plt  # plots

from sklearn.model_selection import StratifiedKFold  # CV
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve  # metrics
from sklearn.pipeline import Pipeline  # pipeline
from sklearn.compose import ColumnTransformer  # column-wise transforms
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # enc/scale
from sklearn.impute import SimpleImputer  # missing
from sklearn.linear_model import LogisticRegression  # LR
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier  # RF/ET
from sklearn.svm import LinearSVC  # SVM base
from sklearn.calibration import CalibratedClassifierCV  # probability for SVM

warnings.filterwarnings("ignore")  # keep logs clean

# ---------- Paths ----------
DATA_PATH = os.path.expanduser("/Users/KrisLiu/Downloads/patient_features_final.csv")  # input csv
OUT_DIR   = os.path.expanduser("/Users/KrisLiu/Downloads/multi_target_eval")  # outputs
FIG_DIR   = os.path.join(OUT_DIR, "figures")  # fig dir
TAB_DIR   = os.path.join(OUT_DIR, "tables")  # table dir
FLIST_DIR = os.path.join(OUT_DIR, "feature_lists")  # feature lists dir
for d in [OUT_DIR, FIG_DIR, TAB_DIR, FLIST_DIR]: os.makedirs(d, exist_ok=True)  # ensure dirs

assert os.path.exists(DATA_PATH), f"File not found: {DATA_PATH}"  # guard
df = pd.read_csv(DATA_PATH)  # load data
print(f"Loaded: {DATA_PATH} | shape={df.shape}")  # status

# ---------- Candidate targets (meters) ----------
# We will try these meters if they exist; threshold = 0.80 by default.                                   
TARGET_SPECS = {
    "n_encounters": 0.80,
    "n_procedures": 0.80,
    "overall_stay_days_mean": 0.80,
    "overall_stay_days_max": 0.80,
    "total_transfers": 0.80,
    "n_distinct_event_types": 0.80,
    "rx_orders_per_encounter_avg": 0.80,
    "rx_admin_per_encounter_avg": 0.80,
}
TARGETS = [c for c in TARGET_SPECS.keys() if c in df.columns]  # keep existing
print("Targets to evaluate:", TARGETS)  # log

# ---------- Entry-time feature policy ----------
ENTRY_MODE = "strict"  # choose from {"strict","liberal"}  # strict uses fewer, safer features at intake

# words that likely indicate post-visit outcomes or heavy utilization signals; drop in strict mode        
STRICT_DROP_PATTERNS = [
    r"\bstay\b", r"length", r"hours", r"encounter", r"procedure", r"transfer", r"rx_", r"admin",
    r"util", r"admit", r"discharge", r"icu", r"vent", r"mort", r"readmit"
]
# whitelist that is safe at entry (demographics / chronic dx buckets etc.)                                 
STRICT_KEEP_HINTS = [
    "age", "sex", "gender", "race", "ethnic", "zip", "payer", "comorb", "chronic", "history",
    "icd_prefix_", "has_", "smoker", "diabetes", "hypertension", "copd", "asthma"
]

def select_entry_features(df_cols, mode="strict"):
    cols = list(df_cols)  # all columns
    # strip obvious non-features                                                                                       
    drop_hard = {"patient_id", "PATIENT_BIRTH_YEAR"}  # ids/raw
    cols = [c for c in cols if c not in drop_hard]  # drop ids
    # split dtypes later; here just names                                                                               
    if mode == "strict":
        # keep if any keep-hint matches or it's categorical-like object                                                 
        keep_cols = []
        for c in cols:
            name = c.lower()
            if df[c].dtype == "object":  # categorical at intake is OK                                                 
                keep_cols.append(c)
            elif any(k in name for k in STRICT_KEEP_HINTS):
                keep_cols.append(c)
        # remove any that match drop patterns                                                                           
        pat = re.compile("|".join(STRICT_DROP_PATTERNS), flags=re.I)
        keep_cols = [c for c in keep_cols if pat.search(c) is None]
        return keep_cols
    else:
        # liberal: use everything except the obviously post-outcome signals; still remove drop patterns                 
        pat = re.compile("|".join(STRICT_DROP_PATTERNS), flags=re.I)
        return [c for c in cols if pat.search(c) is None]

ENTRY_FEATURES_ALL = select_entry_features(df.columns, ENTRY_MODE)  # base feature list
# We will further remove the current target and any columns that leak about that target (same-root names).             

# ---------- Helper: probabilistic OneHotEncoder signature compatibility ----------
from sklearn.preprocessing import OneHotEncoder
onehot_kwargs = {"handle_unknown":"ignore"}  # base kwargs
if "sparse_output" in inspect.signature(OneHotEncoder).parameters:
    onehot_kwargs["sparse_output"] = True  # new api
else:
    onehot_kwargs["sparse"] = True  # old api

# ---------- Models ----------
def build_models():
    models = {}
    # Logistic Regression                                                                                           
    models["LogReg"] = LogisticRegression(
        solver="saga", penalty="l2", class_weight="balanced",
        max_iter=5000, n_jobs=-1, random_state=42
    )
    # Random Forest                                                                                                  
    models["RandForest"] = RandomForestClassifier(
        n_estimators=400, min_samples_leaf=2, class_weight="balanced_subsample",
        n_jobs=-1, random_state=42
    )
    # Extra Trees                                                                                                     
    models["ExtraTrees"] = ExtraTreesClassifier(
        n_estimators=600, min_samples_leaf=2, class_weight="balanced",
        n_jobs=-1, random_state=42
    )
    # Linear SVM with probability calibration                                                                         
    base_svm = LinearSVC(dual=False, class_weight="balanced", random_state=42)
    models["LinearSVM_cal"] = CalibratedClassifierCV(base_svm, method="sigmoid", cv=3)
    # Optional LightGBM                                                                                              
    try:
        import lightgbm as lgb
        models["LightGBM"] = lgb.LGBMClassifier(
            n_estimators=800, learning_rate=0.05, num_leaves=64, min_child_samples=20,
            subsample=0.8, colsample_bytree=0.8, objective="binary", random_state=42, n_jobs=-1
        )
    except Exception:
        pass
    return models

# ---------- Build preprocess pipeline ----------
def make_preprocess(num_cols, cat_cols):
    num_tf = Pipeline([("imp", SimpleImputer(strategy="median")),
                       ("scaler", StandardScaler(with_mean=False))])  # keep sparse compatibility               
    cat_tf = Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                       ("oh", OneHotEncoder(**onehot_kwargs))])  # OHE                                           
    return ColumnTransformer([("num", num_tf, num_cols), ("cat", cat_tf, cat_cols)])

# ---------- Labeling helper ----------
def make_binary_label(series, q=0.80):
    # Use integer ceil for count-like columns; else raw quantile threshold                                          
    thr = series.quantile(q)
    if pd.api.types.is_integer_dtype(series.dtype) or series.name.startswith("n_"):
        thr_int = int(np.ceil(thr))
        y = (series >= thr_int).astype(int)
        info = dict(threshold=thr, threshold_used=thr_int, rule=">= ceil(p{})".format(int(q*100)))
    else:
        y = (series >= thr).astype(int)
        info = dict(threshold=thr, threshold_used=float(thr), rule=">= p{}".format(int(q*100)))
    return y, info

# ---------- Leakage scrub for a given target ----------
def scrub_leakage(base_features, target_name):
    # remove target itself and any column whose name contains the root token(s) of target                             
    toks = re.split(r"[_:]+", target_name.lower())
    toks = [t for t in toks if len(t) >= 3]  # short tokens are noisy                                                
    def leak(c):
        lc = c.lower()
        if c == target_name: return True
        return any(t in lc for t in toks)
    feats = [c for c in base_features if not leak(c)]
    return feats

# ---------- Evaluation per target ----------
CV = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold CV
all_rows = []  # accumulate rows for mega table

for tgt in TARGETS:
    # Build label                                                                                                   
    y, lab_info = make_binary_label(df[tgt], q=TARGET_SPECS[tgt])
    prevalence = float(y.mean())
    # Choose entry-safe features and scrub leakage                                                                    
    feats0 = [c for c in ENTRY_FEATURES_ALL if c != tgt and c in df.columns]  # base entry set
    feats  = scrub_leakage(feats0, tgt)  # remove lookalikes
    # Split dtypes                                                                                                   
    cat_cols = [c for c in feats if df[c].dtype == "object"]
    num_cols = [c for c in feats if c not in cat_cols]
    # Save feature list                                                                                               
    pd.Series(feats).to_csv(os.path.join(FLIST_DIR, f"entry_features_{ENTRY_MODE}_{tgt}.csv"),
                            index=False, header=False)
    # Preprocess                                                                                                      
    preprocess = make_preprocess(num_cols, cat_cols)
    # Models                                                                                                          
    models = build_models()

    print(f"\n=== Target: {tgt} | prev={prevalence:.3%} | rule {lab_info['rule']} with used threshold={lab_info['threshold_used']} ===")
    # Evaluate each model by CV                                                                                       
    for mname, base_est in models.items():
        pipe = Pipeline([("prep", preprocess), ("clf", base_est)])  # full pipeline
        aucs, auprcs = [], []
        # CV loop                                                                                                     
        for tr, te in CV.split(df[feats], y):
            Xtr, Xte = df.iloc[tr][feats], df.iloc[te][feats]
            ytr, yte = y.iloc[tr], y.iloc[te]
            pipe.fit(Xtr, ytr)
            # probabilities or decision
            clf = pipe.named_steps["clf"]
            if hasattr(clf, "predict_proba"):
                prob = pipe.predict_proba(Xte)[:, 1]
            else:
                # Calibrated SVM exposes predict_proba; bare LinearSVC won't reach here                               
                dec = pipe.decision_function(Xte)
                # min-max squash to [0,1] as fallback                                                                 
                m, M = dec.min(), dec.max()
                prob = (dec - m) / (M - m + 1e-9)
            aucs.append(roc_auc_score(yte, prob))
            auprcs.append(average_precision_score(yte, prob))

        row = dict(target=tgt, prevalence=prevalence, threshold_used=lab_info["threshold_used"],
                   model=mname, AUROC=np.mean(aucs), AUROC_std=np.std(aucs),
                   AUPRC=np.mean(auprcs), AUPRC_std=np.std(auprcs),
                   n_features=len(feats), entry_mode=ENTRY_MODE)
        all_rows.append(row)
        print(f"{mname:14s} | AUROC {np.mean(aucs):.3f}±{np.std(aucs):.3f} | AUPRC {np.mean(auprcs):.3f}±{np.std(auprcs):.3f} | feats={len(feats)}")

    # For the best model by AUROC, make OOF-ish plots (train on 4 folds, test on 1, stitched)                         
    best = max([r for r in all_rows if r["target"]==tgt], key=lambda x: x["AUROC"])
    best_name = best["model"]
    best_est = build_models()[best_name]
    pipe_best = Pipeline([("prep", preprocess), ("clf", best_est)])
    # Accumulate OOF predictions                                                                                      
    oof = np.zeros(len(y), dtype=float)
    for tr, te in CV.split(df[feats], y):
        pipe_best.fit(df.iloc[tr][feats], y.iloc[tr])
        clf = pipe_best.named_steps["clf"]
        if hasattr(clf, "predict_proba"):
            oof[te] = pipe_best.predict_proba(df.iloc[te][feats])[:,1]
        else:
            dec = pipe_best.decision_function(df.iloc[te][feats])
            m, M = dec.min(), dec.max()
            oof[te] = (dec - m) / (M - m + 1e-9)
    # Plot ROC/PR                                                                                                      
    fpr, tpr, _ = roc_curve(y, oof)
    prec, rec, _ = precision_recall_curve(y, oof)
    auroc = roc_auc_score(y, oof)
    auprc = average_precision_score(y, oof)

    plt.figure(figsize=(5.2,4.4))
    plt.plot(fpr, tpr, label=f"{best_name} (AUROC={auroc:.3f})")
    plt.plot([0,1],[0,1],"--", lw=1)
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC — {tgt} ({ENTRY_MODE})"); plt.legend(); plt.grid(True, alpha=0.3)
    roc_path = os.path.join(FIG_DIR, f"roc_{tgt}_{best_name}_{ENTRY_MODE}.png")
    plt.tight_layout(); plt.savefig(roc_path, dpi=150); plt.close()
    print("Saved:", roc_path)

    plt.figure(figsize=(5.2,4.4))
    plt.plot(rec, prec, label=f"{best_name} (AUPRC={auprc:.3f})")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR — {tgt} ({ENTRY_MODE})"); plt.legend(); plt.grid(True, alpha=0.3)
    pr_path = os.path.join(FIG_DIR, f"pr_{tgt}_{best_name}_{ENTRY_MODE}.png")
    plt.tight_layout(); plt.savefig(pr_path, dpi=150); plt.close()
    print("Saved:", pr_path)

# ---------- Save mega-table ----------
mega = pd.DataFrame(all_rows).sort_values(["target","AUROC"], ascending=[True,False])
mega.to_csv(os.path.join(TAB_DIR, f"mega_auc_table_{ENTRY_MODE}.csv"), index=False)
print("\nWrote mega table:", os.path.join(TAB_DIR, f"mega_auc_table_{ENTRY_MODE}.csv"))


Loaded: /Users/KrisLiu/Downloads/patient_features_final.csv | shape=(16369, 69)
Targets to evaluate: ['n_encounters', 'n_procedures', 'overall_stay_days_mean', 'overall_stay_days_max', 'total_transfers', 'n_distinct_event_types', 'rx_orders_per_encounter_avg', 'rx_admin_per_encounter_avg']

=== Target: n_encounters | prev=20.698% | rule >= ceil(p80) with used threshold=34 ===
LogReg         | AUROC 0.914±0.003 | AUPRC 0.789±0.012 | feats=54
RandForest     | AUROC 0.858±0.001 | AUPRC 0.632±0.008 | feats=54
ExtraTrees     | AUROC 0.857±0.007 | AUPRC 0.686±0.015 | feats=54
LinearSVM_cal  | AUROC 0.913±0.003 | AUPRC 0.789±0.012 | feats=54
Saved: /Users/KrisLiu/Downloads/multi_target_eval/figures/roc_n_encounters_LogReg_strict.png
Saved: /Users/KrisLiu/Downloads/multi_target_eval/figures/pr_n_encounters_LogReg_strict.png

=== Target: n_procedures | prev=20.637% | rule >= ceil(p80) with used threshold=35 ===
LogReg         | AUROC 0.804±0.013 | AUPRC 0.545±0.032 | feats=54
RandForest     | A